# **Proyecto**
## **Integrantes:**

 - Ruben Acosta Arzate 
 - Jesus Israel Gutierrez Elizalde
 - Erick Martínez Piza 
 - Angel Mauricio López Miranda 
 - Jhony Brandon Lazcano Lagos

## **Descripción:**

Nuestro proyecto consiste en crear una IA que pueda jugar Conecta 4 a un nivel que pueda competir con jugadores promedio. 

El objetivo es encontrar una manera para que la IA decida una buena jugada en cada turno del juego, tomando en cuenta la información del tablero.



# **Descripción del dataset:**

Para resolver nuestro problema utilizamos algoritmos de búsqueda por lo que la información que se requiere al tomar las decisiones para las jugadas son generadas por los algoritmos. Siendo esta la razón de que no requiera datasets.


# **Descripción de los algoritmos propuestos para la solución:**

El primer requerimiento para resolver el problema fue una **heurística**, que dado una tirada y un tablero, ésta nos devolviera un valor. Este valor incrementa de acuerdo a la cantidad de fichas que se conectan (del jugador que tira) tras la jugada.

Lo anterior no es suficiente para crear una buena IA que juegue conecta 4, pues no se toma mucho en cuenta las jugadas que podría hacer el rival dentro de uno, dos o hasta tres turnos. Esto nos lleva a requerir el siguiente algoritmo, **breadth first search**, el cual utilizamos para que dado un tablero se generen las jugadas futuras (un arbol de tableros) a partir de ese tablero. Con la informacion obtenida es posible actualizar nuestra **heurística** para luego, de forma **greedy**, tomar las mejores decisiones.

Por lo que diseñamos una clase ***connect4*** que modela el juego y el tablero; y una clase ***Node*** que representa los nodos del arbol que usa para las posibles jugadas futuras. Los nodos tienen de atributos principales un tablero, una jugada y uno con el nombre de **heurística** que nos sirve para evaluar que tan buena fue la jugada, y a su vez, como factor de decisión.

Mas adelante se hablará en detalle sobre lo anterior.

# **Conecta 4**
Antes de mostrar nuestra solución primero mostraremos lo mas relevante de nuestro código para modelar el juego de conecta 4:



### **Tablero**
De las cosas a notar en la clase `connect4`, es que modelamos el tablero como una lista de listas de $6 \times 7$, y el constructor de los objetos de la 
clase puede crear tableros para juegos nuevos o con jugadas ya hechas, esto si se le pasa como parámetro un tablero. También se tiene un atributo para saber quien es el ganador o si nadie ha ganado aún.

```python
def __init__(self, new_game: bool = True, game_board: list = []):
        """Constructor of the connect4."""
        assert type(game_board) == list
        self.winner = 0
        self.__board = []
        if not new_game:
            assert len(game_board) == 6
            for row in game_board:
                assert len(row) == 7
                for cell in row:
                    assert type(cell) == int and cell > -1 and cell < 3
            self.__board = game_board
            return
        for i in range(6):
            self.__board.append([0, 0, 0, 0, 0, 0, 0])
```

Un ejemplo de como se ve un tablero vacío:
```
    0 1 2 3 4 5 6
    _____________
0 | 0 0 0 0 0 0 0 
1 | 0 0 0 0 0 0 0 
2 | 0 0 0 0 0 0 0 
3 | 0 0 0 0 0 0 0 
4 | 0 0 0 0 0 0 0 
5 | 0 0 0 0 0 0 0 
```

## **Haciendo jugadas**

Para realizar jugadas en el tablero hicimos la siguiente función:
```python
def make_play(self, player_one: bool, column: int) -> bool:
        """Function to make a play on the board.
        Returns True if the play could not be made."""
        if type(player_one) != bool or type(column) != int:
            raise Exception("Wrong argument.")
        if 0 > column or column > 6:
            raise Exception("Wrong column.")

        row = 5
        while self.__board[row][column] != 0:
            row -= 1
            if row < 0:
                return True

        if player_one:
            self.__board[row][column] = 1
        else:
            self.__board[row][column] = 2
        return False
```
Ésta posiciona la ficha del jugador en el lugar indicado del tablero dada una columna. Y en caso de que la jugada sea inválida se devuelve `True`.

Otra función importante es `possible_plays`, la cual dada una instancia de juego nos devuelva todas las jugadas disponibles, es decir, todas las columnas donde todavía se puede tirar:
```python
def possible_plays(self) -> list:
        """Function to get all possible plays."""
        plays = []
        count = 0
        for cell in self.__board[0]:
            if cell == 0:
                plays.append(count)
            count += 1
        return plays
```

La función devuelve una lista con los números representantes de las columnas libres.


## **Determinando ganador**

Hicimos una función que dada una instancia del juego determina si alguien ya ha ganado, y también nos devuelve a ese alguien. Esta función escanea el tablero
en todas las direcciones en las que puede haber una jugada ganadora. Va llevando la cuenta de las fichas contiguas del mismo jugador, para así saber en qué momento alguien ya ganó:

```python
def finished(self) -> int:
        """Checks the board for a winner. If there is a winnner, 
        returns the player number of the winner, otherwise returns 0."""
        (current, contiguous) = (0, 0)
        # check for win by a horizontal play
        for row in range(6):
            (current, contiguous) = (0, 0)
            for column in range(7):
                (current, contiguous) = self.__check_contiguous(
                    row, column, current, contiguous)
                if contiguous == 4:
                    self.winner = current
                    return current
        # check for win by a vertical play
        for column in range(7):
            (current, contiguous) = (0, 0)
            for row in range(6):
                (current, contiguous) = self.__check_contiguous(
                    row, column, current, contiguous)
                if contiguous == 4:
                    self.winner = current
                    return current
        # check for win by a diagonal play (/)
        l = [(0, 3), (0, 4), (0, 5), (0, 6), (1, 6), (2, 6)]
        for i in l:
            (row, column) = i
            (current, contiguous) = (0, 0)
            while row < 6 and column > -1:
                (current, contiguous) = self.__check_contiguous(
                    row, column, current, contiguous)
                if contiguous == 4:
                    self.winner = current
                    return current
                row += 1
                column -= 1
        # check for win by a diagonal play (/)
        l = [(2, 0), (1, 0), (0, 0), (0, 1), (0, 2), (0, 3)]
        for i in l:
            (row, column) = i
            (current, contiguous) = (0, 0)
            while row < 6 and column < 7:
                (current, contiguous) = self.__check_contiguous(
                    row, column, current, contiguous)
                if contiguous == 4:
                    self.winner = current
                    return current
                row += 1
                column += 1


        return 0
```

En caso de que no haya ningún ganador, devuelve cero. Ésta utiliza otra función auxiliar para llevar la cuenta de las fichas contiguas en una misma dirección del jugador que se este revisando:

```python
def __check_contiguous(self, row, column, current, contiguous) -> tuple:
        """Auxiliary function to check winning plays."""
        cell = self.__board[row][column]
        if cell == 0:
            return (0, 0)
        if cell != current:
            return (cell, 1)
        return (current, contiguous + 1)
```

## **Clase completa de conecta 4:**

In [ ]:
import copy


class Connect4:
    """Class to model a connect4 board."""

    def __init__(self, new_game: bool = True, game_board: list = []):
        """Constructor of the connect4."""
        assert type(game_board) == list
        self.winner = 0
        self.__board = []
        if not new_game:
            assert len(game_board) == 6
            for row in game_board:
                assert len(row) == 7
                for cell in row:
                    assert type(cell) == int and cell > -1 and cell < 3
            self.__board = game_board
            return
        for i in range(6):
            self.__board.append([0, 0, 0, 0, 0, 0, 0])

    def __str__(self) -> str:
        """str function of a connect4."""
        string = ""
        n = 0
        string += "    0 1 2 3 4 5 6\n"
        string += "    _____________\n"
        for row in self.__board:
            string += str(n) + " | "
            n += 1
            for cell in row:
                string += str(cell) + " "
            string += "\n"
        return string

    def __eq__(self, connect4) -> bool:
        """Equals function to compare two connect4."""
        if type(self) != type(connect4):
            return False
        if self.winner != connect4.winner:
            return False
        for i in range(6):
            for j in range(7):
                if self.__board[i][j] != connect4.__board[i][j]:
                    return False
        return True

    def make_play(self, player_one: bool, column: int) -> bool:
        """Function to make a play on the board.
        Returns True if the play could not be made."""
        if type(player_one) != bool or type(column) != int:
            raise Exception("Wrong argument.")
        if 0 > column or column > 6:
            raise Exception("Wrong column.")

        row = 5
        while self.__board[row][column] != 0:
            row -= 1
            if row < 0:
                return True

        if player_one:
            self.__board[row][column] = 1
        else:
            self.__board[row][column] = 2
        return False

    def get_board(self) -> list:
        """Returns a copy of the board"""
        return copy.deepcopy(self.__board)

    def finished(self) -> int:
        """Checks the board for a winner. If there is a winnner, 
        returns the player number of the winner, otherwise returns 0."""
        (current, contiguous) = (0, 0)
        # check for win by a horizontal play
        for row in range(6):
            (current, contiguous) = (0, 0)
            for column in range(7):
                (current, contiguous) = self.__check_contiguous(
                    row, column, current, contiguous)
                if contiguous == 4:
                    self.winner = current
                    return current
        # check for win by a vertical play
        for column in range(7):
            (current, contiguous) = (0, 0)
            for row in range(6):
                (current, contiguous) = self.__check_contiguous(
                    row, column, current, contiguous)
                if contiguous == 4:
                    self.winner = current
                    return current
        # check for win by a diagonal play (/)
        l = [(0, 3), (0, 4), (0, 5), (0, 6), (1, 6), (2, 6)]
        for i in l:
            (row, column) = i
            (current, contiguous) = (0, 0)
            while row < 6 and column > -1:
                (current, contiguous) = self.__check_contiguous(
                    row, column, current, contiguous)
                if contiguous == 4:
                    self.winner = current
                    return current
                row += 1
                column -= 1
        # check for win by a diagonal play (/)
        l = [(2, 0), (1, 0), (0, 0), (0, 1), (0, 2), (0, 3)]
        for i in l:
            (row, column) = i
            (current, contiguous) = (0, 0)
            while row < 6 and column < 7:
                (current, contiguous) = self.__check_contiguous(
                    row, column, current, contiguous)
                if contiguous == 4:
                    self.winner = current
                    return current
                row += 1
                column += 1


        return 0

    def __check_contiguous(self, row, column, current, contiguous) -> tuple:
        """Auxiliary function to check winning plays."""
        cell = self.__board[row][column]
        if cell == 0:
            return (0, 0)
        if cell != current:
            return (cell, 1)
        return (current, contiguous + 1)

    def possible_plays(self) -> list:
        """Function to get all possible plays."""
        plays = []
        count = 0
        for cell in self.__board[0]:
            if cell == 0:
                plays.append(count)
            count += 1
        return plays

# **Nodos del arbol**
El mecanismo usado para que nuestra IA decida cual es la mejor jugada fue a través de un árbol. En él revisamos un número suficiente de posibles jugadas futuras, y con apoyo de una heurística, el programa puede tomar una buena decisión.

En esta sección revisamos a fondo la implementación y los algoritmos que utilizamos.

## **Nodo**

Cada nodo tiene los siguientes atributos:
- Una instancia del juego. 
- Una lista para sus hijos. 
- Un atributo para saber quien fue el jugador que hizo la jugada para llegar a esa instancia del juego (la que guarda el nodo).
- Un apuntador a su padre.
- Un atributo para guardar su heurística.
- Un atributo para saber la altura del nodo en el árbol

```python
def __init__(self, connect4, current_player: int, parent=None, height: int = 0) -> None:
        """Constructor of a node"""
        assert type(connect4) == Connect4.connect4
        assert type(current_player) == int
        assert height >= 0
        assert current_player == 1 or current_player == 2
        self.__connect4 = Connect4.connect4(False, connect4.get_board())
        self.__children = []
        self.__current_player = current_player
        self.__parent = parent
        self.__heuristic = 0
        self.__height = height
```

## **Expandiendo nodos**
Para expandir algun nodos primero verificamos que no haya ganador del la instancia del juego que se encuentra en el nodo que queremos expandir. En caso de que haya ganador no se puede expandir ese nodo. 

Luego obtenemos al siguiente jugador utilizando el jugador del nodo actual.

Despues, por cada jugada posible que hay en la instancia del juego que tiene el nodo que estamos expandiendo, creamos una copia de esa instancia y luego realizamos la jugada en la copia.

Creamos un nodo con esa copia que ahora tiene la nueva jugada, le aplicamos la heurística y guardamos ese valor.

Por ultimo agregamos ese nuevo nodo a la lista de hijos del nodo que se expandio.

```python
def expand(self) -> None:
        """Function to expand this node. Won't expand finished games"""
        if self.__connect4.finished() != 0:
            return
        next_player = 1 if self.__current_player == 2 else 2
        for play in self.__connect4.possible_plays():
            copy = Connect4.connect4(False, self.__connect4.get_board())
            copy.make_play(True if next_player == 1 else False, play)
            new_node = node(copy, next_player, self, self.__height + 1)
            new_node.__heuristic = self.heuristic(new_node, play)
            self.__children.append((new_node, play))
```

## **Heuristica de un nodo**

Para calcular que tan buena fue una jugada utilizamos una función que recibe un nodo y una jugada (columna donde se tiro).

Con el nodo y la jugada se puede obtener quien es el jugador que hizo la jugada y exactamente donde se encuentra la ficha que tiro.

Si sucede que el jugador que hizo la jugada gano entonces devolvemos un numero muy grande, en caso contrario llamamos a una funcion auxiliar que da valor a la jugada dependiendo de cuantas fichas se logro conectar con la jugada.

Esta funcion da mas puntos mientras conecte cadenas de fichas mas largas, en la misma direccion claro, es decir, da mas puntos si conecta 3 en lugar de 2. Esta funcion va en todas las direcciones para contar fichas conectadas.

```python
def __check_contiguous(self, row: int, column: int, player: int, board: list) -> int:
        """Aux private function to check for contiguous chips of the player."""
        contiguous = 0
        # down
        mult = 1
        curr_row = row + 1
        while curr_row < 6 and board[curr_row][column] == player:
            contiguous += mult
            mult += 4
            curr_row += 1

        # left
        mult = 1
        curr_col = column - 1
        while curr_col > -1 and board[row][curr_col] == player:
            contiguous += mult
            mult += 4
            curr_col -= 1

        # right
        mult = 1
        curr_col = column + 1
        while curr_col < 7 and board[row][curr_col] == player:
            contiguous += mult
            mult += 4
            curr_col += 1

        # up-right
        mult = 1
        curr_row = row - 1
        curr_col = column + 1
        while curr_col < 7 and curr_row > -1 and board[curr_row][curr_col] == player:
            contiguous += mult
            mult += 4
            curr_row -= 1
            curr_col += 1

        # up-left
        mult = 1
        curr_row = row - 1
        curr_col = column - 1
        while curr_col > -1 and curr_row > -1 and board[curr_row][curr_col] == player:
            contiguous += mult
            mult += 4
            curr_row -= 1
            curr_col -= 1

        # down-right
        mult = 1
        curr_row = row + 1
        curr_col = column + 1
        while curr_col < 7 and curr_row < 6 and board[curr_row][curr_col] == player:
            contiguous += mult
            mult += 4
            curr_row += 1
            curr_col += 1

        # down-left
        mult = 1
        curr_row = row + 1
        curr_col = column - 1
        while curr_col > -1 and curr_row < 6 and board[curr_row][curr_col] == player:
            contiguous += mult
            mult += 4
            curr_row += 1
            curr_col -= 1
        return contiguous
```

La funcion de la heuristica nos queda:

```python
def heuristic(self, next_node, move):
        """Heuristic of a play, the better the play the higher value."""
        board = next_node.__connect4.get_board()
        row = 0
        while row < 5 and board[row][move] == 0:
            row += 1
        player = board[row][move]
        assert player != 0
        if next_node.__connect4.finished() == player:
            return 1000000000
        return self.__check_contiguous(row, move, player, board)
```

## **Creando el árbol**

Para crear el árbol utilizaremos el algoritmo **breadth first search** pero lo limitaremos a cierta altura pues hay muchos tableros de juego sin ganador y seria demasiado tardado calcularlos todos.

Estamos limitando nuestro arbol a que sea un arbol de altura 4, tomando la raiz con altura 0. Podria parecer que es muy poco pero con esta altura y suponiendo que en la instancia del juego siguen todas la jugadas disponibles, el arbol tendria 2402 nodos. 

Luego de que generamos el arbol de altura 4 para un nodo falta algo muy importante, que es actualizar las heuristicas. 

Para esto lo que hacemos es que a los nodos padres les restamos una parte de la heuristica de sus hijos. Esto lo hacemos de las hojas hacia la raiz.

Esto anterior es muy importante pues es claro que una jugada buena del rival significa que posiblemente yo tuve una mala jugada.

Y es justo siguiendo esta logica anterior por lo que realizamos la actualización de heuristicas de los nodos.

```python
def height_4_tree(self) -> None:
        """This node is taken as root and generates a tree of all the posible
        moves 4 shifts ahead using a limited (by height) breadth first search.
        """
        queue = [self]
        expected_height = self.__height + 4
        nodes = {self.__height: [self], self.__height +
                 1: [], self.__height + 2: [], self.__height + 3: []}
        while queue != []:
            curr_node = queue.pop(0)
            if curr_node.get_children() == []:
                curr_node.expand()
            for (child, _) in curr_node.get_children():
                if child.__height < expected_height:
                    queue.append(child)
                    nodes[child.__height].append(child)

        levels = [3, 2, 1, 0]
        for level in levels:
            # update heuristic of nodes of that level
            for inner_node in nodes[self.__height + level]:
                for (child, _) in inner_node.get_children():
                    inner_node.__heuristic -= child.__heuristic * .2
```


## **Clase completa para nodos**

In [ ]:
class Node:
    """Class to model the nodes for the search model."""

    def __init__(self, connect4, current_player: int, parent=None, height: int = 0) -> None:
        """Constructor of a node"""
        assert type(connect4) == type(Connect4())
        assert type(current_player) == int
        assert height >= 0
        assert current_player == 1 or current_player == 2
        self.__connect4 = Connect4(False, connect4.get_board())
        self.__children = []
        self.__current_player = current_player
        self.__parent = parent
        self.__heuristic = 0
        self.__height = height

    def __str__(self) -> str:
        return str(self.__connect4) + "\nplayer: " + str(self.__current_player) + ", height: " + str(self.__height) + ", heuristic: " + str(self.__heuristic) + "\n\n"

    def expand(self) -> None:
        """Function to expand this node. Won't expand finished games"""
        if self.__connect4.finished() != 0:
            return
        next_player = 1 if self.__current_player == 2 else 2
        for play in self.__connect4.possible_plays():
            copy = Connect4(False, self.__connect4.get_board())
            copy.make_play(True if next_player == 1 else False, play)
            new_node = Node(copy, next_player, self, self.__height + 1)
            new_node.__heuristic = self.heuristic(new_node, play)
            self.__children.append((new_node, play))

    def heuristic(self, next_node, move):
        """Heuristic of a play, the better the play the higher value."""
        board = next_node.__connect4.get_board()
        row = 0
        while row < 5 and board[row][move] == 0:
            row += 1
        player = board[row][move]
        assert player != 0
        if next_node.__connect4.finished() == player:
            return 1000000000
        return self.__check_contiguous(row, move, player, board)

    def __check_contiguous(self, row: int, column: int, player: int, board: list) -> int:
        """Aux private function to check for contiguous chips of the player."""
        contiguous = 0
        # down
        mult = 1
        curr_row = row + 1
        while curr_row < 6 and board[curr_row][column] == player:
            contiguous += mult
            mult += 4
            curr_row += 1

        # left
        mult = 1
        curr_col = column - 1
        while curr_col > -1 and board[row][curr_col] == player:
            contiguous += mult
            mult += 4
            curr_col -= 1

        # right
        mult = 1
        curr_col = column + 1
        while curr_col < 7 and board[row][curr_col] == player:
            contiguous += mult
            mult += 4
            curr_col += 1

        # up-right
        mult = 1
        curr_row = row - 1
        curr_col = column + 1
        while curr_col < 7 and curr_row > -1 and board[curr_row][curr_col] == player:
            contiguous += mult
            mult += 4
            curr_row -= 1
            curr_col += 1

        # up-left
        mult = 1
        curr_row = row - 1
        curr_col = column - 1
        while curr_col > -1 and curr_row > -1 and board[curr_row][curr_col] == player:
            contiguous += mult
            mult += 4
            curr_row -= 1
            curr_col -= 1

        # down-right
        mult = 1
        curr_row = row + 1
        curr_col = column + 1
        while curr_col < 7 and curr_row < 6 and board[curr_row][curr_col] == player:
            contiguous += mult
            mult += 4
            curr_row += 1
            curr_col += 1

        # down-left
        mult = 1
        curr_row = row + 1
        curr_col = column - 1
        while curr_col > -1 and curr_row < 6 and board[curr_row][curr_col] == player:
            contiguous += mult
            mult += 4
            curr_row += 1
            curr_col -= 1
        return contiguous

    def get_children(self) -> list:
        """Function to get the children of this node."""
        return self.__children

    def get_height(self) -> int:
        """Function to get the height of this node."""
        return self.__height

    def get_parent(self):
        """Function to get the parent of this node."""
        return self.__parent

    def get_heuristic(self) -> int:
        """Function to get the heuristic of this node."""
        return self.__heuristic

    def get_current_player(self) -> int:
        """Function to get the current player of this node."""
        return self.__current_player

    def get_board(self) -> list:
        """Function to get the board of this node."""
        return self.__connect4.get_board()

    def height_4_tree(self) -> None:
        """This node is taken as root and generates a tree of all the posible
        moves 4 shifts ahead using a limited (by height) breadth first search.
        """
        queue = [self]
        expected_height = self.__height + 4
        nodes = {self.__height: [self], self.__height +
                 1: [], self.__height + 2: [], self.__height + 3: []}
        while queue != []:
            curr_node = queue.pop(0)
            if curr_node.get_children() == []:
                curr_node.expand()
            for (child, _) in curr_node.get_children():
                if child.__height < expected_height:
                    queue.append(child)
                    nodes[child.__height].append(child)

        levels = [3, 2, 1, 0]
        for level in levels:
            # update heuristic of nodes of that level
            for inner_node in nodes[self.__height + level]:
                for (child, _) in inner_node.get_children():
                    inner_node.__heuristic -= child.__heuristic * .2

    def get_all_leaves(self) -> list:
        """Returns all the leaves of this subtree."""
        leaves = []
        queue = []
        queue.append(self)
        while queue != []:
            curr_node = queue.pop(0)
            if curr_node.get_children() == []:
                leaves.append(curr_node)
                continue
            for child in curr_node.get_children():
                queue.append(child[0])
        return leaves

# **Evaluación de los resultados**

Consideremos la siguiente instancia del juego:

In [ ]:
prueba = Connect4()
prueba.make_play(True, 3)
prueba.make_play(True, 4)
prueba.make_play(False, 3)
print(prueba)

    0 1 2 3 4 5 6
    _____________
0 | 0 0 0 0 0 0 0 
1 | 0 0 0 0 0 0 0 
2 | 0 0 0 0 0 0 0 
3 | 0 0 0 0 0 0 0 
4 | 0 0 0 2 0 0 0 
5 | 0 0 0 1 1 0 0 



Es claro que si la IA no tira en la columna 2 o 5 estaría regalando el juego, esto debido a que si después el usuario tira en esas columnas tendría 3 en línea horizontal, creando dos posibles lugares para conectar 4 en su siguiente turno. Y como la IA solo puede tirar una ficha por turno, no es posible tapar esas dos pisibilidades.

In [ ]:
nodo_prueba = Node(prueba, 1) # creamos un nodo con el juego anterior
nodo_prueba.height_4_tree() # generamos el arbol 
hijos = nodo_prueba.get_children() # obtenemos a los hijos
mejor_heuristica = hijos[0][0].get_heuristic()
mejor_hijo = hijos[0][0]
for (hijo, _) in hijos:
  if hijo.get_heuristic() > mejor_hijo.get_heuristic():
    mejor_hijo = hijo
print(mejor_hijo)

    0 1 2 3 4 5 6
    _____________
0 | 0 0 0 0 0 0 0 
1 | 0 0 0 0 0 0 0 
2 | 0 0 0 0 0 0 0 
3 | 0 0 0 0 0 0 0 
4 | 0 0 0 2 0 0 0 
5 | 0 0 2 1 1 0 0 

player: 2, height: 1, heuristic: -96000002.80000001




Observemos que como la IA tomará la que tenga mayor heurística, tirará su ficha en la columna 2.

La forma que evaluamos los resultadaos fue jugando contra la IA. Más adelante proporcionaremos el código con el que se puede tener una partida contra ella.

La IA entra en acción justo después de que el usuario realiza una jugada. Ésta se ve reflejada en la instancia del juego.

Una vez hecha la jugada, se creará el árbol con esa instancia y la IA tomará la mejor decisión de acuerdo al árbol creada y la heurística obtenida e cada una de sus ramas.



In [ ]:
def user_input() -> int:
    """Function to get a valid play from the user."""
    not_finished = True
    while not_finished:
        usr_input = input()
        usr_int = -1
        try:
            usr_int = int(usr_input)
        except:
            print("Must enter a number!!\n\nTry again: ",end="")
            continue
        if usr_int > 6 or usr_int < 0:
            print("Invalid number!!\n\nTry again: ",end="")
            continue
        not_finished = False 

    return usr_int 

game = Connect4()
curr_node = Node(game, 2)
curr_node.expand()

player1 = True
print(game)
while game.finished() == 0 and game.possible_plays() != []:
    play = None
    if player1:
        print("player1 make a play: ", end="")
        play = user_input() 
        print()
    else:
        # children is a list of tuples, (child, play)
        children = curr_node.get_children()
        (curr_best_heuristic, curr_best_play) = (children[0][0].get_heuristic(), children[0][1])
        for (child, curr_play) in children:
            if child.get_heuristic() > curr_best_heuristic:
                curr_best_heuristic = child.get_heuristic()
                curr_best_play = curr_play
        play = curr_best_play


    if game.make_play(player1, play):
        print(" Invalid play!!!!!!!!\n")
        continue
    print(game)
    if player1:
        print("\nAI making a move")
        curr_node = Node(game, 1)
        curr_node.height_4_tree()
        player1 = False
    else:
        player1 = True
    print()

print(game)
if game.winner == 0:
    print("There was a tie.")
else:    
    print("The winner is player " + str(game.winner))

# **Conclusiones**

Con los resultados y pruebas realizadas, podemos decir que nuestro programa juega bastante bien. Queda claro que no esta diseñada para ser la mejor del mundo y es muy dificil que sea derrotada, pero un descuido por parte del usuario no será perdonado y la IA se llevará la victoria.

Consideramos que el **breadth first search** limitado por altura y nuestra **heuristica** fueron herramientas muy buenas para resolver el problema. 